In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import io
import os
import warnings
pd.set_option('display.max_columns', 500)

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import SelectPercentile, RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, cross_validate, cross_val_score, RepeatedStratifiedKFold,\
StratifiedKFold
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder, RobustScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, PrecisionRecallDisplay, precision_recall_curve, confusion_matrix, recall_score, precision_score
from sklearn.exceptions import ConvergenceWarning
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler, SMOTE
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
import category_encoders as ce
from scipy import stats
from scipy.stats import chi2_contingency
from datetime import datetime, date
import optuna
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Introduction

The Small Business Administration (SBA) was founded in 1953 to assist small businesses in obtaining loans. Small businesses have been the primary source of employment in the United States. Helping small businesses help with job creation, which reduces unemployment. Small business growth also promotes economic growth. One of the ways the SBA helps small businesses is by guaranteeing bank loans. This guarantee reduces the risk to banks and encourages them to lend to small businesses. If the loan defaults, the SBA covers the amount guaranteed, and the bank suffers a loss for the remaining balance.

There have been several small business success stories like FedEx and Apple. However, the rate of default is very high. Many economists believe the banking market works better without the assistance of the SBA. Supporter claim that the social benefits and job creation outweigh any financial costs to the government in defaulted loans.

## What to do
In this project, we will attempt to assist the Small Business Administration (SBA) in making decisions about whether a company is eligible for a loan based on the company's demographic data.

## The Goal

To develop a robust and data-driven decision-making system that leverages company demographic data to accurately assess and determine the eligibility of businesses for loans, ultimately aiding the Small Business Administration (SBA) in making informed lending decisions and promoting responsible financial support for small enterprises. For this time, we will focus on 

## The Values

The project's primary value lies in enhancing the efficiency and accuracy of the loan approval process for small businesses. By leveraging data-driven insights and analytics, it empowers the Small Business Administration (SBA) to:

1. **Minimize Risk**: Reduce the likelihood of providing loans to businesses with a high risk of default, safeguarding financial resources.

2. **Support Growth**: Facilitate access to capital for deserving small businesses, promoting growth and economic development.

3. **Data-Driven Decisions**: Foster a culture of data-driven decision-making within the SBA, leading to more informed and equitable lending practices.

4. **Cost-Efficiency**: Optimize resource allocation by focusing on businesses most likely to succeed, lowering administrative and operational costs.

5. **Economic Impact**: Contribute to the success and sustainability of small businesses, positively impacting local economies and job creation.

6. **Transparency**: Increase transparency in the lending process, building trust among applicants and stakeholders.

In summary, the project's value is realized through improved loan approval outcomes, reduced risk exposure, and the promotion of financial inclusivity and responsible lending practices in support of small business success.

## The Dataset

The original data set is from the U.S.SBA loan database, which includes historical data from 1987 through 2014 (899,164 observations) with 27 variables. The data set includes information on whether the loan was paid off in full or if the SMA had to charge off any amount and how much that amount was.

For more information on this data set go to https://amstat.tandfonline.com/doi/full/10.1080/10691898.2018.1434342

## Constain
For the sake of learning and due to computation limitation, we will only focus on California State, since it is the most state there are.

In [ ]:
sba = pd.read_csv('/kaggle/input/should-this-loan-be-approved-or-denied/SBAnational.csv')
sba.head()

In [ ]:
sba.shape

In [ ]:
def report(df):
    col = []
    d_type = []
    uniques = []
    n_uniques = []
    nan = []

    for i in df.columns:
        col.append(i)
        d_type.append(df[i].dtypes)
        uniques.append(df[i].unique()[:5])
        n_uniques.append(df[i].nunique())
        nan.append(df[i].isna().sum()/len(df)*100)

    return pd.DataFrame({'Column': col, 'd_type': d_type, 'unique_sample': uniques, 'n_uniques': n_uniques, 'nan%': nan})

report(sba)

In [ ]:
print(f'Object columns: {len(sba.select_dtypes(include="object").columns)}')
print(f'Numeric columns: {len(sba.select_dtypes(exclude="object").columns)}')

# Data Cleansing and Preprocessing

## ChgOffDate

In [ ]:
# drop ChgOffDate because it has to many nan values
sba.drop('ChgOffDate', axis=1, inplace=True)

## State

In [ ]:
# focus on top 5 states only (California, Texas, New York, Florida, 'Pennsylvania')
top_5_state = sba['State'].value_counts(normalize=True).sort_values(ascending=False).head().index
sba5 = sba[sba['State']=='CA']
sba5['State'].unique()

In [ ]:
sba5.drop('State', axis=1, inplace=True)

## MIS_Status

In [ ]:
sba5['MIS_Status'].isna().sum()

In [ ]:
sba5.dropna(subset='MIS_Status', inplace=True)
sba5['MIS_Status'].isna().sum()

In [ ]:
# checking MIS Status values
sba5['MIS_Status'].value_counts(normalize=True)

In [ ]:
# changing it to integer
pd.Series(np.where(sba5['MIS_Status']=='P I F', 0, 1)).value_counts(normalize=True)

In [ ]:
sba5['MIS_Status'] = np.where(sba5['MIS_Status']=='P I F', 0, 1)

In [ ]:
# check duplicated rows based on all columns
sba5.duplicated().sum()

## ApprovalDate

In [ ]:
# Get month of approval date and delete ApprovalDate because it's too detail information
sba5['ApprovalMonth'] = sba5['ApprovalDate'].str.split('-').str[1]
sba5.drop('ApprovalDate', axis=1, inplace=True)

## NAICS

In [ ]:
# convert NAICS code into related sector
sba5['NAICS'] = sba5['NAICS'].astype('string')

naics_to_sector = {
    '11': 'Agriculture, Forestry, Fishing and Hunting',
    '21': 'Mining, Quarrying, and Oil and Gas Extraction',
    '22': 'Utilities',
    '23': 'Construction',
    '31-33': 'Manufacturing',
    '42': 'Wholesale Trade',
    '44-45': 'Retail Trade',
    '48-49': 'Transportation and Warehousing',
    '51': 'Information',
    '52': 'Finance and Insurance',
    '53': 'Real Estate and Rental and Leasing',
    '54': 'Professional, Scientific, and Technical Services',
    '55': 'Management of Companies and Enterprises',
    '56': 'Administrative and Support and Waste Management and Remediation Services',
    '61': 'Educational Services',
    '62': 'Health Care and Social Assistance',
    '71': 'Arts, Entertainment, and Recreation',
    '72': 'Accommodation and Food Services',
    '81': 'Other Services (except Public Administration)',
    '92': 'Public Administration',
    # Add more NAICS codes and sectors as needed
}

# Function to get the sector for a given NAICS code
def get_sector(naics_code):
    # Handle cases where the code has more than 2 digits (e.g., '31-33')
    first_two = naics_code[:2]
    if first_two in ['31', '44', '48']:
        if first_two == '31':
            return naics_to_sector['31-33']
        elif first_two == '44':
            return naics_to_sector['44-45']
        else:
            return naics_to_sector['48-49']
    else:
        return naics_to_sector.get(first_two, 'Unknown Sector')
    
sba5['Sector'] = sba5['NAICS'].apply(get_sector)
sba5.drop('NAICS', axis=1, inplace=True)

## Term

In [ ]:
# Process Term into group
(sba5['Term']/365).describe()

In [ ]:
sba5['TermGroup'] = np.where(sba5['Term']<=90,'Below 3 months',
                            np.where((sba5['Term']>90) & (sba5['Term']<=180), '3-6 months',
                            np.where((sba5['Term']>180) & (sba5['Term']<=365), '6-12 months', 'More Than a Year')))
display(sba5[['Term', 'TermGroup']].sample(10))
sba5.drop('Term', axis=1, inplace=True)

## NewExist

In [ ]:
# drop NewExist when 0 and NaN
display(sba5['NewExist'].value_counts(dropna=False))
sba5 = sba5[sba5['NewExist']!=0]
sba5.dropna(subset='NewExist')['NewExist'].value_counts(dropna=False)

In [ ]:
# drop NaN from NewExist. Will not effect that much to the model.
sba5 = sba5.dropna(subset='NewExist')

In [ ]:
sba5['NewExist'] = np.where(sba5['NewExist']==1, 'Exist Business', 'New Business')

## BalanceGross

In [ ]:
# drop BalanceGross it has only one value
sba5.drop('BalanceGross', axis=1, inplace=True)

## ChgOffPrinGr

In [ ]:
# drop ChgOffPrinGr it will leaks information to the model.
# Everytime it has a value, the status always CHGOFF
sba5.drop('ChgOffPrinGr', axis=1, inplace=True)

## SBA_Appv

In [ ]:
# drop SBA_Appv, in the future will would not know how much the amount will be approved
sba5.drop('SBA_Appv', axis=1, inplace=True)

## DisbursementGross

In [ ]:
# drop DisbursementGross, we don't need it
sba5.drop('DisbursementGross', axis=1, inplace=True)

## DisbursementDate

In [ ]:
# drop DisbursementDate, we don't need it
sba5.drop('DisbursementDate', axis=1, inplace=True)

## ApprovalFY

In [ ]:
# We drop approvalFY besauce we want to predict future loan proposal.
# So, if we want to predict proposal from FY 2023, the model will not know the data yet.
sba5.drop('ApprovalFY', axis=1, inplace=True)

## GrAppv

In [ ]:
# process GrAppv, replace $ and , with empty string
sba5['GrAppv'] = sba5['GrAppv'].str.replace('$', '')
sba5['GrAppv'] = sba5['GrAppv'].str.replace(',', '')
sba5['GrAppv'] = sba5['GrAppv'].astype('float')

## Name

In [ ]:
checking_name = sba5.groupby(['Name', 'NewExist']).agg({"Name":"count", "MIS_Status":"mean"}).rename(columns={'Name':'Count', 'NewExist': 'NewExist_Ratio', 'MIS_Status':'CHGOff_Ratio'})
checking_name.sort_values('Count', ascending=False)

Some companies have borrowed quite frequently. The company that borrows the most frequently is Subway. In total, only 6% or about 21 out of 342 loans they took out resulted in a Charge Off, while the rest were Paid in Full (PIF). When still considered a new company, Subway always made an effort to pay off their loans, so when they became a more established company, they were trusted more and found it easier to obtain loans.

Having a low Charge Off (loan default) rate in the early stages can reflect responsible credit behavior. Here's an interpretation of your observation:

1. **High Borrowing Frequency**: Subway appears to be active in utilizing loan facilities. Out of 342 loans, only about 6% resulted in Charge Offs. This indicates that Subway generally manages its debts responsibly.

2. **Good Credit Quality**: The fact that Subway tends to repay its loans when it's still considered a new company shows good credit quality. This could help build a positive reputation with lenders, making it easier for them to obtain loan approvals in the future.

3. **Trustworthiness**: Companies that consistently repay loans on time or even ahead of schedule have a better chance of establishing a reputation as a financially trustworthy entity. This can assist Subway in gaining easier access to loans and other sources of funding as they become more established.

4. **Importance of Credit History**: A positive credit history is crucial in the business world. Having a positive track record in terms of loan repayments can help companies secure better loan terms, including lower interest rates.

It's important to remember that Subway's credit behavior can serve as a positive example for other companies. Implementing prudent financial policies and maintaining good credit quality can lead to long-term financial benefits and business growth.

In [ ]:
sba5.info()

## RevLineCr

In [ ]:
# checking RevLineCr
sba5['RevLineCr'].value_counts()

In [ ]:
# We will filter only N and Y since those are only explainable value
sba5 = sba5[sba5['RevLineCr'].isin(['N','Y'])]

## LowDoc

In [ ]:
# checking LowDoc
sba5['LowDoc'].value_counts()

In [ ]:
# We will filter only N and Y since those are only explainable value
sba5 = sba5[sba5['LowDoc'].isin(['N','Y'])]

## Bank, BankState, City, LowDoc

In [ ]:
# We drop rows that the Bank, BankState, City and LowDoc that have Null value
sba5.dropna(subset=['Bank', 'BankState', 'City', 'LowDoc'], inplace=True)

## BankState

In [ ]:
sba5['BankState'].value_counts().describe()

In [ ]:
# We convert BankState that appeared under 15 times into 'OTHER'
count_bankst = sba5['BankState'].value_counts().to_frame().reset_index().rename(columns={'count':'BankstCount'})
sba5 = sba5.merge(count_bankst, on='BankState', how='left')
sba5['NewBankState'] = np.where(sba5['BankstCount']>15, sba5['BankState'], 'OTHER')
sba5.drop(['BankState', 'BankstCount'], axis=1, inplace=True)

## Name, LoanNr_ChkDgt

In [ ]:
sba5.drop(['Name', 'LoanNr_ChkDgt', 'Zip'], axis=1, inplace=True)

## FranchiseCode

In [ ]:
sba5['IsFranchise'] = np.where(((sba5['FranchiseCode']==1) | (sba5['FranchiseCode']==0)), 'Y', 'N')
sba5.drop('FranchiseCode', axis=1, inplace=True)

In [ ]:
sba5['IsFranchise'].value_counts()

In [ ]:
sba5.info()

## City

In [ ]:
# City has too many unique value.
# Also, most of them (75%) are only appear once or 12 times only.
# We need to process it so it would not make the model too complex.
sba5['City'] = sba5['City'].str.upper()
sba5['City'].nunique()

In [ ]:
sba5['City'].value_counts().to_frame().describe()

In [ ]:
count_city = sba5['City'].value_counts().to_frame().reset_index().rename(columns={'count': 'City_Count'})

In [ ]:
count_city[(count_city['City_Count']>25)&(count_city['City_Count']<=50)]

In [ ]:
sba5 = sba5.merge(count_city, on='City')

In [ ]:
sba5['NewCity'] = np.where(sba5['City_Count']>100, sba5['City'], 'OTHER')

In [ ]:
checking_city = sba5.groupby('NewCity').agg({'MIS_Status':'mean'}).sort_values('MIS_Status', ascending=False).reset_index()
checking_city[checking_city['NewCity']=='OTHER']

In [ ]:
sba5.drop(['City', 'City_Count'],axis=1,inplace=True)

## Bank

In [ ]:
# Bank has too many unique value.
# Also, most of them (75%) are only appear once or 29 times only.
# We need to process it so it would not make the model too complex.
sba5['Bank'] = sba5['Bank'].str.upper()
sba5['Bank'].nunique()

In [ ]:
sba5['Bank'].value_counts().to_frame().describe()

In [ ]:
count_bank = sba5['Bank'].value_counts().to_frame().reset_index().rename(columns={'count': 'Bank_Count'})

In [ ]:
count_bank[(count_bank['Bank_Count']>1)&(count_bank['Bank_Count']<=29)]

In [ ]:
sba5 = sba5.merge(count_bank, on='Bank')
sba5['NewBank'] = np.where(sba5['Bank_Count']>100, sba5['Bank'], 'OTHER')
checking_bank = sba5.groupby('NewBank').agg({'MIS_Status':'mean'}).sort_values('MIS_Status', ascending=False).reset_index()
checking_bank[checking_bank['NewBank']=='OTHER']

In [ ]:
sba5.drop(['Bank', 'Bank_Count'], axis=1, inplace=True)

In [ ]:
sba5['MIS_Status'].value_counts()

In [ ]:
sba5['UrbanRural'] = np.where(sba5['UrbanRural']==1, 'Urban',
                             np.where(sba5['UrbanRural']==2, 'Rural', 'Undefined'))

## GrAppv

In [ ]:
# we don't need to use this feature since we are needed to help banks to determine
# the company's status (CHGOFF or PIF) even before the bank approves their proposal
sba5.drop('GrAppv', axis=1, inplace=True)

# Exploratory Data Analysis and Visualization

In [ ]:
sba5.rename(columns={'MIS_Status':'ChargeOff'}, inplace=True)
sba5['ChargeOff'] = sba5['ChargeOff'].astype('category')

In [ ]:
sba_cat = sba5.select_dtypes(include='object')
sba_num = sba5.select_dtypes(exclude='object')

In [ ]:
for col in sba_cat.columns:
    sba_cat[col] = sba_cat[col].astype('category')

In [ ]:
sba_cat.columns

In [ ]:
sba_cat.head()

In [ ]:
def cat_vs_chgoff(column):
    chgoff_ = sba5.groupby(column)['ChargeOff'].value_counts(normalize=True).to_frame()
    chgoff_.columns = ['Proportion']
    chgoff_pvt = chgoff_.reset_index().\
    pivot(columns='ChargeOff', index=column, values='Proportion')
    chgoff_pvt.fillna(0, inplace=True)
    display(chgoff_pvt*100)

    contingency = sba5.groupby([column, 'ChargeOff']).agg({'ChargeOff':'count'}).rename(columns={'ChargeOff':'Count'}).reset_index().pivot(index=column, columns='ChargeOff', values='Count')
    stat, p, dof, expected = chi2_contingency(contingency)
    if(p<0.05):
        print(f'Column {column} has strong relationship with ChargeOff.')
    else:
        print(f'Column {column} has weak relationship with ChargeOff.')

    chgoff_pvt.plot(kind='bar', stacked=True, figsize=(16,10))
    plt.title(f'Charge Off proportion by {column}')
    plt.show()
    print('------------------------------------------------------------------------------')

In [ ]:
for col in sba_cat.columns:
    cat_vs_chgoff(col)

In [ ]:
def num_vs_chgoff(column):
    data1 = sba_num[sba5['ChargeOff']==0][column]
    data2 = sba_num[sba5['ChargeOff']==1][column]
    
    stat, p = stats.ttest_ind(a=data1, b=data2, equal_var=True)
    if(p<0.05):
        print(f'Column {column} has strong relationship with MIS Status.')
    else:
        print(f'Column {column} has weak relationship with MIS Status.')

    plt.figure(figsize=(10,5))
    plt.title(f'{col} distribution Split by ChargeOff')
    sns.boxplot(data=sba5, y='ChargeOff', x=column)
    plt.legend()
    plt.show()
    print('------------------------------------------------------------------------------')

In [ ]:
sba_num.head()

In [ ]:
num_cols = list(sba_num.columns)
num_cols.remove('ChargeOff')
for col in num_cols:
    num_vs_chgoff(col)

## Correlation Between Independent Variables

In [ ]:
sns.heatmap(sba_num.corr(), annot=True)
plt.show()

## VIF (Multicolinearity Check)

In [ ]:
vif_data = pd.DataFrame()
vif_data["feature"] = sba_num.drop('ChargeOff', axis=1).columns
vif_data["VIF"] = [vif(sba_num.drop('ChargeOff', axis=1).values, i)
                          for i in range(len(sba_num.drop('ChargeOff', axis=1).columns))]
vif_data

In [ ]:
vif_data = pd.DataFrame()
vif_data["feature"] = sba_num.drop(['CreateJob', 'ChargeOff'], axis=1).columns
vif_data["VIF"] = [vif(sba_num.drop(['CreateJob', 'ChargeOff'], axis=1).values, i)
                          for i in range(len(sba_num.drop(['CreateJob', 'ChargeOff'], axis=1).columns))]
vif_data

## Average ChargeOff

In [ ]:
checking_diff = sba[sba['MIS_Status']=='CHGOFF'][['Name', 'ChgOffPrinGr', 'GrAppv']].dropna()
checking_diff.head()

In [ ]:
checking_diff['ChgOffPrinGr'] = checking_diff['ChgOffPrinGr'].str.replace('$','').str.replace(',','').astype('float')
checking_diff['GrAppv'] = checking_diff['GrAppv'].str.replace('$','').str.replace(',','').astype('float')

diff_by_name = checking_diff.groupby('Name').agg({'ChgOffPrinGr':'max', 'GrAppv':'sum'})
diff_by_name['Diff%'] = diff_by_name['ChgOffPrinGr']/diff_by_name['GrAppv']*100
diff_by_name.describe()

In [ ]:
diff_by_name[diff_by_name['Diff%']>100]

In [ ]:
print(f'Object columns: {len(sba5.select_dtypes(include="object").columns)}')
print(f'Numeric columns: {len(sba5.select_dtypes(exclude="object").columns)}')

# Machine Learning Modelling
For the current machine learning modeling, we aim to minimize the model's prediction of companies that will actually Charge Off but are predicted as Paid in Full (PIF). Hence, we will focus on Recall Score.

"Charge Off" is a condition where a lender determines that a debt is unlikely to be collected and treats it as a loss on their financial records. This would result in losses for the borrowers.

According to the data we currently have, the Charge Off rate stands at 72.4%. This means that if a company has a debt of USD 1,000,000, it cannot pay USD 724,000 to the lender.

## Strategy
In the modeling step, I will use a Pipeline to help me build the model. First of all, I will divide the data into two categories:

1. Categorical Variables
2. Quantitative Variables

A categorical variable is data that takes category or label values. On the other hand, quantitative variables take numerical values and represent some kind of measurement.

There are 13 columns that I have listed as categorical variables. Since each column values has no order with each other, I will encode them using **One-Hot Encoding**.

The rest of the columns should be listed in the other category, and since each column has a significantly different value range from each other and highly skewed, I will try to scale them using **Robust Scaler**.

Since there are no missing values, I don't need an imputer in this process.

The models that I will use is **Logistic Regression** and **Random Forest Classifier**. We will compare both of them and choose the model which give us the best result.

In [ ]:
sba5.info()

In [ ]:
X = sba5.drop(columns='ChargeOff')
y = sba5['ChargeOff']

X_train_val, X_test, y_train_val, y_test = train_test_split(X,y, stratify=y, random_state=2023)

In [ ]:
X_train_val.shape, X_test.shape

## Logistic Regression Pipeline

In [ ]:
one_hot_cols = X.select_dtypes(include='object').columns
numeric_cols = X.select_dtypes(exclude='object').columns
logit = LogisticRegression(solver='liblinear', random_state=2023)
smote = SMOTE(random_state=2023)

logit_pipe_num = Pipeline([
    ('scaler', RobustScaler()),
])

# for all object columns
logit_pipe_cat = Pipeline([
    ('onehot', OneHotEncoder(drop='first')),
])

# transforming all columns
logit_transformer = ColumnTransformer([
    ('pipe_num', logit_pipe_num, numeric_cols),
    ('pipe_cat', logit_pipe_cat, one_hot_cols)
])

# combine all pipeline
logit_pipe_combine = Pipeline([
    ('transformer', logit_transformer),
    ('rfe', RFE(logit)),
    ('resampling', smote),
    ('logit', logit)
])

## Random Forest Pipeline

In [ ]:
rfc = RandomForestClassifier(max_depth=7, min_samples_split=10, random_state=2023)
smote = SMOTE(random_state=2023)

rfc_pipe_num = Pipeline([
    ('scaler', RobustScaler()),
])

# for all object columns
rfc_pipe_cat = Pipeline([
    ('onehot', OneHotEncoder(drop='first')),
])

# transforming all columns
rfc_transformer = ColumnTransformer([
    ('pipe_num', rfc_pipe_num, numeric_cols),
    ('pipe_cat', rfc_pipe_cat, one_hot_cols)
])

# combine all pipeline
rfc_pipe_combine = Pipeline([
    ('transformer', rfc_transformer),
    ('rfe', RFE(rfc)),
    ('resampling', smote),
    ('rfc', rfc)
])

## Model Benchmark

In [ ]:
start_time = datetime.now()
skf = StratifiedKFold(n_splits=5)

logit_scoreres = cross_validate(logit_pipe_combine, X_train_val, y_train_val, scoring=['recall', 'precision'], cv=skf, n_jobs=-1, verbose=1, return_train_score=True)
rfc_scoreres = cross_validate(rfc_pipe_combine, X_train_val, y_train_val, scoring=['recall', 'precision'], cv=skf, n_jobs=-1, verbose=1, return_train_score=True)

recall_res = [
    np.mean(logit_scoreres['test_recall']),
    np.mean(logit_scoreres['train_recall']),
    np.mean(rfc_scoreres['test_recall']),
    np.mean(rfc_scoreres['train_recall'])
]
precision_res = [
    np.mean(logit_scoreres['test_precision']),
    np.mean(logit_scoreres['train_precision']),
    np.mean(rfc_scoreres['test_precision']),
    np.mean(rfc_scoreres['train_precision'])
]
compare = pd.DataFrame({'model':[logit_pipe_combine['logit'],logit_pipe_combine['logit'],rfc_pipe_combine['rfc'],rfc_pipe_combine['rfc']],
                        'data' :['TEST', 'TRAIN', 'TEST', 'TRAIN'],
                        'recall': recall_res,
                       'precision': precision_res})

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
display(compare)

In [ ]:
# tuning logistic regression
def lr_optimize(trial):
    params = {
        'max_iter': trial.suggest_int("max_iter",100,500),
        'C': trial.suggest_float('C', 1, 2.5),
        'solver': trial.suggest_categorical('solver', ['liblinear']),
        'penalty': trial.suggest_categorical('penalty', ['l1', 'l2']),
    }

    # logistic regression
    logit_pipe_combine = Pipeline([
    ('transformer', logit_transformer),
    ('rfe', RFE(logit)),
    ('resampling', smote),
    ('logit', LogisticRegression(**params, random_state=2023))
    ])
    
    inner_cv = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 2023)
    inner_scores = cross_val_score(logit_pipe_combine, X_train_val, y_train_val, cv=inner_cv, scoring='recall', n_jobs=4)
    
    cv_score = np.mean(inner_scores)
    return cv_score

In [ ]:
# # Execute the tuning
# study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=2023), study_name='LogitSBALoanApproval 1.0')
# study.optimize(lr_optimize, n_trials=5)

In [ ]:
# find out the best parameter
# best_params = study.best_params
best_params = {'max_iter': 156, 'C': 1.7018433884122988, 'solver': 'liblinear', 'penalty': 'l2'}
print(best_params)

In [ ]:
logit_best = LogisticRegression(**best_params, random_state=2023)
    
logit_pipe_combine = Pipeline([
('transformer', logit_transformer),
('rfe', RFE(logit)),
('resampling', smote),
('logit', logit_best)
])

logit_pipe_combine.fit(X_train_val, y_train_val)

In [ ]:
recall_score(y_test,logit_pipe_combine.predict(X_test))

In [ ]:
display_ = PrecisionRecallDisplay.from_estimator(
    logit_pipe_combine, X_test, y_test, name="Logit")
_ = display_.ax_.set_title("2-class Precision-Recall curve")

In [ ]:
y_scores = logit_pipe_combine.predict_proba(X_test)[:,1]
precision, recall, thresholds = precision_recall_curve(y_test.values, y_scores)
df_thres = pd.DataFrame({'Precision': precision[:-1], 'Recall': recall[:-1], 'Threshold': thresholds})
df_thres[df_thres['Recall'].between(.85, .9)]

# Evaluation

## Confusion Matrix

In [ ]:
pred_40 = np.where(y_scores>0.403914, 1, 0)
confusion_matrix(y_test,pred_40)

## Recall

In [ ]:
recall_40 = recall_score(y_test,pred_40)
recall_40

## Precision

In [ ]:
precision_40 = precision_score(y_test,pred_40)
precision_40

# Conclusion:

Up to this point, we have been able to improve the Recall to 89%. This means that out of 100 companies applying for loans and actually being potential defaulters, we can accurately predict 89 of these companies. The remaining 11 companies, we mispredict as non-defaulters, and we continue to lend money to them.

If we lend 100,000 USD to these 11 companies and, based on our previous analysis, the average charge-off rate is around 72.4%, then each company will Charge Off approximately 79,640 USD, resulting in a total default amount of 796,400 USD.

However, without using machine learning, we have limitations in assessing whether a company will default or not. At best, we can predict 50% accurately. In the same context, this would result in a loss of 3,620,000 USD from defaulting companies.

Here we can see that by using Machine Learning, we can reduce the risk of losses when providing capital loans.